In [52]:
import numpy as np
import matplotlib.pyplot as plt
import re

# used to look at images
from skimage import io, color, filters, feature
from skimage.transform import resize, rotate
from PIL import Image, ImageOps

# to read in the .mat files
import scipy.io as sio 


# tensorflow - for CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model

# used to import the image folders
from tensorflow.keras.preprocessing import image, image_dataset_from_directory

from tensorflow.keras.models import Model
from keras.layers.convolutional import *
from keras.layers.normalization import BatchNormalization

from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping

### Used the train_list.mat file to create dictionary of label names and class names

In [2]:
train_dict = sio.loadmat('extras/lists/train_list.mat')

In [3]:
# use regex to create list of breed names from train_dict
breeds = []
for i in np.unique(train_dict['labels']):
    # get one file - first 0 is just first file with the label
    # second 0 is just the file path
    file = train_dict['file_list'][train_dict['labels'] == i][0][0]
    # regex pattern
    # n[0-9]+- denotes a pattern that starts with n, has the group 0-9 an unknown amount of times and a hyphen
    # OR a pattern that starts with non-whitespace (\W for the forward slash), n[0-9]+, an underscore
    # [0-9] some amount of times and ends with .jpg
    pattern = re.compile(r'n[0-9]+-|\Wn[0-9]+_[0-9]+.jpg')
    # replace this pattern for each file with an empty string and append to breeds list
    breeds.append(pattern.sub('', file))

In [4]:
# create empty dictionary for labels
breed_key = dict()
for i in np.unique(train_dict['labels']):
    # labels start at 1, so to match python idx, subtract 1 from breeds
    breed_key[i] = breeds[i-1]
    
# breed_key

### Load in the images and place them into the appropriate batches

In [5]:
train_path1 = 'extras/Images/train1'
train_path2 = 'extras/Images/train2'
train_path3 = 'extras/Images/train3'
train_path4 = 'extras/Images/train4'
train_path5 = 'extras/Images/train5'
# train_path = 'extras/Images_copy/train'

valid_path = 'extras/Images/valid'
test_path = 'extras/Images/test'

## Creating VGG16 model

In [56]:
# transfer learning model
inceptionresnetv2_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [57]:
# freeze layers that have already been trained
for layer in inceptionresnetv2_model.layers[:-1]:
    layer.trainable = False

In [60]:
# adding new model on top
inputs = Input(shape = (224, 224, 3))
x = inceptionresnetv2_model(inputs, training = False)
# converts features of shape of model's amount of rows to vectors
x = GlobalAveragePooling2D()(x)

# create expected output layer - final prediction layer (dense) to predict the 120 classes
output = Dense(120, activation = 'softmax')(x)

# put it all together
analyzer_model_preprocess = Model(inputs, output)

In [49]:
analyzer_model_preprocess.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [62]:
# this is the augementation used for training, testing, and validation data
# consider changing the pixels to be 0-1 in ImageDataGenerator
datagen = image.ImageDataGenerator()
datagen_training = image.ImageDataGenerator(validation_split=0.1, preprocessing_function=preprocess_input)


# train_batch = datagen.flow_from_directory(train_path, target_size = (224, 224),
#                                           classes = breeds, batch_size = 5)


# Make smaller training batches for sake of computer
train_batch1 = datagen_training.flow_from_directory(train_path1, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'training', seed = 42)
train_batch2 = datagen_training.flow_from_directory(train_path2, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'training', seed = 42)
train_batch3 = datagen_training.flow_from_directory(train_path3, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'training', seed = 42)
train_batch4 = datagen_training.flow_from_directory(train_path4, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'training', seed = 42)
train_batch5 = datagen_training.flow_from_directory(train_path5, target_size = (224, 224),
                                          classes = breeds, batch_size = 5,
                                          subset = 'training', seed = 42)

# respective validation batches of training batches
valid_batch1 = datagen_training.flow_from_directory(train_path1, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'validation', seed = 42)
valid_batch2 = datagen_training.flow_from_directory(train_path2, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'validation', seed = 42)
valid_batch3 = datagen_training.flow_from_directory(train_path3, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'validation', seed = 42)
valid_batch4 = datagen_training.flow_from_directory(train_path4, target_size = (224, 224),
                                          classes = breeds, batch_size = 5, 
                                          subset = 'validation', seed = 42)
valid_batch5 = datagen_training.flow_from_directory(train_path5, target_size = (224, 224),
                                          classes = breeds, batch_size = 5,
                                          subset = 'validation', seed = 42)

test_batch = datagen.flow_from_directory(test_path, target_size = (224, 224),
                                        classes = breeds, batch_size = 5)# transfer learning model

Found 3054 images belonging to 120 classes.
Found 3009 images belonging to 120 classes.
Found 3009 images belonging to 120 classes.
Found 3009 images belonging to 120 classes.
Found 3009 images belonging to 120 classes.
Found 272 images belonging to 120 classes.
Found 274 images belonging to 120 classes.
Found 274 images belonging to 120 classes.
Found 274 images belonging to 120 classes.
Found 274 images belonging to 120 classes.
Found 4122 images belonging to 120 classes.


Ideas:
- Change learning rate to 0.001?
- Change CNN Model - InceptionResNetV2

In [63]:
# compile model
# used gradient descent as optimizer
optimizer = Adam(lr = 0.0001)
analyzer_model_preprocess.compile(optimizer = optimizer,
                      loss = 'categorical_crossentropy',
                      metrics = ['accuracy'])

# Stops training if validation loss doesn't improve
callback = EarlyStopping(monitor = 'val_loss', patience = 3)

In [64]:
# batch_size = 32 is good starting point
# steps_per_epoch = training_size / batch_size
analyzer_model_preprocess.fit(train_batch1, validation_data = valid_batch1, epochs = 8, callbacks = callback)

Epoch 1/8
611/611 [==============================] - 626s 1s/step - loss: 3.8281 - accuracy: 0.2782 - val_loss: 1.1136 - val_accuracy: 0.7831
Epoch 2/8
611/611 [==============================] - 701s 1s/step - loss: 0.7661 - accuracy: 0.8596 - val_loss: 0.7294 - val_accuracy: 0.8015
Epoch 3/8
611/611 [==============================] - 707s 1s/step - loss: 0.4662 - accuracy: 0.8751 - val_loss: 0.6719 - val_accuracy: 0.8051
Epoch 4/8
611/611 [==============================] - 626s 1s/step - loss: 0.3575 - accuracy: 0.8977 - val_loss: 0.6656 - val_accuracy: 0.8125
Epoch 5/8
611/611 [==============================] - 525s 858ms/step - loss: 0.2993 - accuracy: 0.9130 - val_loss: 0.6664 - val_accuracy: 0.8051
Epoch 6/8
611/611 [==============================] - 467s 764ms/step - loss: 0.2684 - accuracy: 0.9192 - val_loss: 0.6889 - val_accuracy: 0.7941
Epoch 7/8
611/611 [==============================] - 496s 812ms/step - loss: 0.2321 - accuracy: 0.9294 - val_loss: 0.6840 - val_accuracy: 0.80

In [65]:
analyzer_model_preprocess.fit(train_batch2, validation_data = valid_batch2, epochs = 8, callbacks = callback)

Epoch 1/8
602/602 [==============================] - 484s 804ms/step - loss: 0.5843 - accuracy: 0.8355 - val_loss: 0.5609 - val_accuracy: 0.8467
Epoch 2/8
602/602 [==============================] - 434s 721ms/step - loss: 0.4660 - accuracy: 0.8548 - val_loss: 0.5524 - val_accuracy: 0.8431
Epoch 3/8
602/602 [==============================] - 434s 721ms/step - loss: 0.3981 - accuracy: 0.8744 - val_loss: 0.5431 - val_accuracy: 0.8467
Epoch 4/8
602/602 [==============================] - 432s 718ms/step - loss: 0.3455 - accuracy: 0.8843 - val_loss: 0.5222 - val_accuracy: 0.8431
Epoch 5/8
602/602 [==============================] - 467s 775ms/step - loss: 0.3048 - accuracy: 0.8933 - val_loss: 0.5596 - val_accuracy: 0.8321
Epoch 6/8
602/602 [==============================] - 440s 731ms/step - loss: 0.2723 - accuracy: 0.9086 - val_loss: 0.5681 - val_accuracy: 0.8248
Epoch 7/8
602/602 [==============================] - 421s 700ms/step - loss: 0.2456 - accuracy: 0.9172 - val_loss: 0.5833 - val_ac

In [66]:
analyzer_model_preprocess.fit(train_batch3, validation_data = valid_batch3, epochs = 8, callbacks = callback)

Epoch 1/8
602/602 [==============================] - 419s 696ms/step - loss: 0.5592 - accuracy: 0.8465 - val_loss: 0.6847 - val_accuracy: 0.8285
Epoch 2/8
602/602 [==============================] - 417s 693ms/step - loss: 0.4454 - accuracy: 0.8681 - val_loss: 0.6486 - val_accuracy: 0.8321
Epoch 3/8
602/602 [==============================] - 416s 691ms/step - loss: 0.3802 - accuracy: 0.8860 - val_loss: 0.6564 - val_accuracy: 0.8358
Epoch 4/8
602/602 [==============================] - 411s 683ms/step - loss: 0.3281 - accuracy: 0.8993 - val_loss: 0.6868 - val_accuracy: 0.8321
Epoch 5/8
602/602 [==============================] - 411s 683ms/step - loss: 0.2937 - accuracy: 0.9059 - val_loss: 0.7027 - val_accuracy: 0.8321


In [67]:
analyzer_model_preprocess.fit(train_batch4, validation_data = valid_batch4, epochs = 8, callbacks = callback)

Epoch 1/8
602/602 [==============================] - 413s 686ms/step - loss: 0.5336 - accuracy: 0.8415 - val_loss: 0.4425 - val_accuracy: 0.8650
Epoch 2/8
602/602 [==============================] - 410s 681ms/step - loss: 0.4334 - accuracy: 0.8594 - val_loss: 0.4251 - val_accuracy: 0.8686
Epoch 3/8
602/602 [==============================] - 410s 682ms/step - loss: 0.3713 - accuracy: 0.8714 - val_loss: 0.4331 - val_accuracy: 0.8577
Epoch 4/8
602/602 [==============================] - 413s 686ms/step - loss: 0.3294 - accuracy: 0.8827 - val_loss: 0.4454 - val_accuracy: 0.8577
Epoch 5/8
602/602 [==============================] - 414s 687ms/step - loss: 0.2946 - accuracy: 0.9006 - val_loss: 0.4418 - val_accuracy: 0.8577


In [68]:
analyzer_model_preprocess.fit(train_batch5, validation_data = valid_batch5, epochs = 8, callbacks = callback)

Epoch 1/8
602/602 [==============================] - 413s 686ms/step - loss: 0.5507 - accuracy: 0.8481 - val_loss: 0.4514 - val_accuracy: 0.8504
Epoch 2/8
602/602 [==============================] - 413s 686ms/step - loss: 0.4382 - accuracy: 0.8691 - val_loss: 0.4025 - val_accuracy: 0.8723
Epoch 3/8
602/602 [==============================] - 412s 684ms/step - loss: 0.3743 - accuracy: 0.8790 - val_loss: 0.4329 - val_accuracy: 0.8796
Epoch 4/8
602/602 [==============================] - 410s 682ms/step - loss: 0.3266 - accuracy: 0.8917 - val_loss: 0.4318 - val_accuracy: 0.8577
Epoch 5/8
602/602 [==============================] - 411s 683ms/step - loss: 0.2914 - accuracy: 0.9050 - val_loss: 0.4300 - val_accuracy: 0.8759


# 